In [24]:
import random
import pandas as pd

from ME_BPER import ME_BPER
from ME_IPER import extract_last_names
from ME_LOC import add_location
from ME_ORG import add_organisation

import sys
sys.path.append("../scripts")

from load_data import mapping, read_tsv_file

In [25]:
df = pd.read_csv("../data_aug_sources/the-middle-east-cities.csv", sep = ";", skiprows = 1)
unique_city_da = df["city_da"].drop_duplicates()
ME_LOC = [add_location(loc) for loc in unique_city_da]

In [26]:
df = pd.read_csv("../data_aug_sources/middle_eastern_organisations.csv", sep = ";", skiprows = 1)
unique_orgs = df["org"].drop_duplicates()
ME_ORG = [add_organisation(org) for org in unique_orgs]

In [27]:
ME_BPER

['Malak',
 'Najib',
 'Abdel',
 'Intisar',
 'Ayman',
 'Anis',
 'Talal',
 'Shahin',
 'Tahsin',
 'Othman',
 'Zakaria',
 'Sajida',
 'Hadi',
 'May',
 'Rayan',
 'Sardar',
 'Nuran',
 'Nasir',
 'Jamilla',
 'Sadik',
 'Hamza',
 'Dalal',
 'Ilyas',
 'Ajmal',
 'Ramadan',
 'Shahnaz',
 'Shakila',
 'Wisam',
 'Omer',
 'Jasmine',
 'Abdalla',
 'Benyamin',
 'Hani',
 'Sarah',
 'Saida',
 'Dahlia',
 'Nasima',
 'Tarik',
 'Malaika',
 'Sharifa',
 'Mahdi',
 'Arif',
 'Kazim',
 'Yassin',
 'Akbar',
 'Ilham',
 'Zaynab',
 'Amar',
 'Seyed',
 'Dani',
 'Mina',
 'Lubna',
 'Fathi',
 'Hamsa',
 'Rehana',
 'Asia',
 'Manna',
 'Youssef',
 'Tuba',
 'Osman',
 'Karam',
 'Yassine',
 'Kasim',
 'Jamshid',
 'Rabia',
 'Bünyamin',
 'Morad',
 'Hanan',
 'Shahrzad',
 'Nawal',
 'Issa',
 'Aladdin',
 'Waqar',
 'Najat',
 'Mostafa',
 'Sawsan',
 'Fatema',
 'Amina',
 'Osama',
 'Jasmin',
 'Maha',
 'Metin',
 'Raja',
 'Mostapha',
 'Marwa',
 'Belal',
 'Idriz',
 'Ayub',
 'Lulu',
 'Khalid',
 'Salem',
 'Hasib',
 'Moussa',
 'Farhan',
 'Said',
 'Zeina',


In [28]:
ME_IPER = extract_last_names("../data_aug_sources/middle_eastern_last_names.txt")
print(ME_IPER)

['Mohamed', 'Hussein', 'Ali', 'Hassan', 'Kazem', 'Abbas', 'Jassim', 'Abdel', 'Ahmed', 'Abdullah', 'Salman', 'Ibrahim', 'Saleh', 'Mahmoud', 'Khalaf', 'Hamid', 'Mahdi', 'Mohsen', 'Khudair', 'Karim', 'Gabr', 'Alwan', 'Jabbar', 'Aboud', 'Obaid', 'Rashid', 'Jawad', 'Nasser', 'Hashem', 'Dawood', 'Ismail', 'Hadi', 'Majid', 'Hamad', 'Fadel', 'Yousef', 'Saeed', 'Abdelhossein', 'Hamoud', 'Mousa', 'Khalil', 'Radi', 'Hamza', 'Qasim', 'Farhan', 'Attieh', 'Hammadi', 'Aziz', 'Lafet', 'Faleh', 'Alawi', 'Eawda', 'Yassin', 'Najm', 'Jaber', 'Salem', 'Naima', 'Hassoun', 'Abdul karim', 'Jaafar', 'Shaker', 'Eisaa', 'Abdul razzaq', 'Abdelraza', 'Katie', 'Faraj', 'Abdelhassen', 'Rahim', 'Jamil', 'Kamel', 'Habib', 'Abdul amir', 'Awad', 'Matar', 'Mansour', 'Taher', 'Abdaly', 'Zaher', 'Mostafa', 'Taha', 'Jumah', 'Zghir', 'Naji', 'Fayad', 'Latif', 'Shehab', 'Rassin', 'Sadiq', 'Abdul jabbar', 'Idan', 'Abdel rahman', 'Sultan', 'Nayef', 'Hamoudi', 'Saad', 'Hatem', 'Khamis', 'Nasif', 'Reda', 'Zidan', 'Khalifa', 'Nour

In [29]:
# getting data 
# path to the data files
path_train = "../data/da_news_train.tsv"
path_dev = "../data/da_news_dev.tsv"
path_test = "../data/da_news_test.tsv"

# create mapping
label2id, id2label = mapping(path_train)

# read in the DaN+ data
train_data = read_tsv_file(path_train, label2id)
dev_data = read_tsv_file(path_dev, label2id)
test_data = read_tsv_file(path_test, label2id)

In [30]:
for sent in dev_data:
    if len(sent["tokens"]) != len(sent["ner_tags"]):
        print("my data is the issue:(:", sent)



In [41]:
def data_aug_replace(dataset):
    for sent in dataset:
        i = 0

        while i<len(sent["tokens"]):
            #print(f"i = {i}, tokens = {len(sent['tokens'])}, tags = {len(sent['ner_tags'])}")
            tag = sent["ner_tags"][i]

            if tag == 'B-PER':
                replace = random.choice(ME_BPER)
                sent["ner_tags"][i] = replace
                ME_BPER.remove(replace)
                i+=1

            elif tag == 'I-PER':
                replace = random.choice(ME_IPER)
                sent["ner_tags"][i] = replace
                ME_IPER.remove(replace)
                i+=1

            elif tag == 'B-LOC':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-LOC":
                    span_len += 1
                    i += 1

                same_length_LOC = []
                for loc in ME_LOC:
                    if len(loc["tokens"]) == span_len:
                        same_length_LOC.append(loc)
                if same_length_LOC:
                    replace = random.choice(same_length_LOC)
                    sent["tokens"][span_start:span_start+span_len] = replace["tokens"]
                    ME_LOC.remove(replace)
            
            elif tag == 'B-ORG':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-ORG":
                    span_len += 1
                    i += 1

                same_length_ORG = []
                for org in ME_ORG:
                    if len(org["tokens"]) == span_len:
                        same_length_ORG.append(org)
                if same_length_ORG:
                    replace = random.choice(same_length_ORG)
                    sent["tokens"][span_start:span_start+span_len] = replace["tokens"]
                    ME_ORG.remove(replace)

            else:
                i+=1

    return dataset

In [42]:
ME = data_aug_replace(dev_data)
print(ME)

[{'tokens': ['Hvor', 'kommer', 'julemanden', 'fra', '?'], 'ner_tags': ['O', 'O', 'O', 'O', 'O'], 'tag_ids': [4, 4, 4, 4, 4]}, {'tokens': ['Flertallet', 'lever', 'stadig', 'under', 'plastikstykker', 'eller', 'tæpper', ',', 'som', 'de', 'har', 'spændt', 'ud', 'over', 'nogle', 'stokke', 'som', 'et', 'improviseret', 'telt', '.'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'tag_ids': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]}, {'tokens': ['Kelds', 'oplæg', 'blev', 'fulgt', '100', 'procent', ',', '"', 'pointerer', 'Susan', 'Mackensie', '.'], 'ner_tags': ['Inas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Waqar', 'Vijayan', 'O'], 'tag_ids': [5, 4, 4, 4, 4, 4, 4, 4, 4, 5, 1, 4]}, {'tokens': ['Raki', 'fortyndes', 'med', 'vand', 'og', 'får', 'et', 'uskyldigt', 'mælkeagtigt', 'udseende', ',', 'som', 'man', 'dog', 'ikke', 'skal', 'lade', 'sig', 'narre', 'af', '!'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 